In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

# 信号前沿分析
pandas pandas_datareader.data  pdr.data.get_data_yahoo()  
pandas dataframe d.asfreq()  
pandas dataframe d.fillna()  
pandas dataframe d.head()  

In [ ]:
names  = ['AAPL','GOOG','MSFT','GS','MS','BAC','C']

df =web.get_data_yahoo(names,'2009/1/1','2012/6/1')
d1 = df['Adj Close']
d1.head()

In [ ]:
# pandas dataframe d.asfreq()
px = pd.DataFrame({n: d1[n] for n in names})
px = px.asfreq(freq='B').fillna(method='pad')
px.head() 

In [ ]:
'''
绘制每只股票的累计收益
pandas dataframe d.pct_change()
pandas dataframe d.add()
pandas dataframe d.cumprod()  
pandas dataframe d.sub()
pandas dataframe d.plot.line()
'''
rets = px.pct_change()
rets.add(1).cumprod().sub(1).plot.line()

In [ ]:
'''
对于投资组合的构建，我们要计算特定回顾期的动量，然后按降序排列并标准化  
python str printf-style String Formatting d
pandas dataframe d.shift() 
pandas dataframe d.rank() ascending 
pandas dataframe d.mean() 
pandas dataframe d.std()
pandas dataframe d.resample() 
pandas Resampling r.first()  
pandas Resampling r.apply() 
pandas dataframe d.mul()  
pandas dataframe d.add()  
pandas dataframe d.prod()
pandas series s.sub()
pandas series s.mean()
pandas series s.std()
numpy Mathematical functions np.sqrt() 
numpy Broadcasting 标量  
'''
def calc_mom(price,lookback,lag):
    mom_ret = price.shift(periods=lag).pct_change(periods=lookback)
    ranks =mom_ret.rank(axis=1,ascending=False)    
    demeaned = ranks.sub(ranks.mean(axis=1),axis=0)    
    return demeaned.div(demeaned.std(axis=1),axis=0)

compound = lambda x: x.add(1).prod().sub(1)
daily_sr =lambda x:x.mean()/x.std()

def strat_sr(prices,lb,hold):
    # 计算投资组合权重
    freq ='%dB' % hold
    port =calc_mom(prices,lb,lag=1)
    daily_rets = prices.pct_change()
    
    # 计算投资组合收益
    port = port.shift(periods=1).resample(rule=freq).first()
    returns =daily_rets.resample(rule=freq).apply(compound)
    port_rets = port.mul(returns).sum(axis=1)
    
    return daily_sr(port_rets)*np.sqrt(252/hold)

# 通过价格数据以及一对参数组合调用该函数将会得到一个标量值
strat_sr(px,70,30)